In [1]:
import pandas as pd
import datetime as dt
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
#python 3.11.5
#selenium==4.24.0
#Flask-SQLAlchemy==2.5.1
#pandas==2.0.3
#bs4==0.0.2
#psycopg2-binary==2.9.9


<h4> Selenium Driver for Firefox </h4>

In [3]:
driver=webdriver.Firefox()

driver.get('https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm')

data = driver.page_source


<h3> bs4 soup </h3>

In [4]:
soup = BeautifulSoup(data, 'html.parser')

soup = soup.find_all('table') # for not duplicating data; more tables were found

In [5]:
# scraping with bs4;

def main(soup, index):

    soup = soup[index]

    row_set = []
    for row in soup.find_all('tr'):
        
        col_set = []

        for td in row.find_all('td'):
            col_set.append(td.text.strip())
        
        # regex to clean the data;
        try:
            col_set[0] = re.search('[a-zA-Z+\t ]+(\w*)' , col_set[0]).group().strip().replace('\t', '')
        except:
            print('hm oops, got some error', col_set)
            
        
        if len(col_set) >= 6: # slicing relevant and irregular data;
            col_set.pop(2)

        row_set.append(col_set)


    # small cleaning 
    
    try:    row_set.remove([])
    except: None

    return row_set


In [6]:
df0 = pd.DataFrame(main(soup, 0)
                  )

df0['ask_bid'] = 'ask'
df0.head(3)

hm oops, got some error []


,0,1,2,3,4,5,ask_bid
0,Tesouro Prefixado 2027,"13,18%","R$ 766,97",01/01/2027,Simule o investimento,None,ask
1,Tesouro Prefixado 2031,"13,24%","R$ 467,73",01/01/2031,Simule o investimento,None,ask
2,Tesouro Prefixado com juros semestrais 2035,"12,97%","R$ 878,82",01/01/2035,Simule o investimento,None,ask


In [7]:
df1 = pd.DataFrame(main(soup, 1)
                   )
df1['ask_bid'] = 'bid'
df1.head(3)

hm oops, got some error []


,0,1,2,3,ask_bid
0,Tesouro Prefixado 2025,"11,39%","R$ 983,44",01/01/2025,bid
1,Tesouro Prefixado 2026,"13,06%","R$ 867,84",01/01/2026,bid
2,Tesouro Prefixado 2027,"13,30%","R$ 765,23",01/01/2027,bid


In [8]:
# joining bid and ask cotations;
df = pd.concat([df0, df1])

In [9]:
# discarting useless cols;

df = df.drop(columns = [4,5])

In [10]:
# renaming cols 

columns = ['titulo', 'taxa', 'pu', 'vencimento', 'ask_bid']
df.columns = columns

In [11]:
df = df.dropna(how = 'all') 

### Cleaning and Transforming

In [12]:
pd.set_option('display.max_rows', 1000)

display(df.head(3))
df.dtypes

,titulo,taxa,pu,vencimento,ask_bid
0,Tesouro Prefixado 2027,"13,18%","R$ 766,97",01/01/2027,ask
1,Tesouro Prefixado 2031,"13,24%","R$ 467,73",01/01/2031,ask
2,Tesouro Prefixado com juros semestrais 2035,"12,97%","R$ 878,82",01/01/2035,ask


titulo        object
taxa          object
pu            object
vencimento    object
ask_bid       object
dtype: object

In [13]:
# numeric dtypes;

df.loc[:, 'pu'] = df.pu.str.replace("R$ ", '')
df.loc[:, 'pu'] = df.pu.str.replace('.', '')
df.loc[:, 'pu'] = df.pu.str.replace(',', '.')
df['pu'] = df.pu.astype(float)

In [14]:
## datetime;

df['vencimento'] = df.vencimento.apply(lambda x: dt.datetime.strptime(x, '%d/%m/%Y'))

In [15]:
display(df.head(3))
df.dtypes

,titulo,taxa,pu,vencimento,ask_bid
0,Tesouro Prefixado 2027,"13,18%",766.97,2027-01-01,ask
1,Tesouro Prefixado 2031,"13,24%",467.73,2031-01-01,ask
2,Tesouro Prefixado com juros semestrais 2035,"12,97%",878.82,2035-01-01,ask


titulo                object
taxa                  object
pu                   float64
vencimento    datetime64[ns]
ask_bid               object
dtype: object

In [16]:
# test if there is any duplicated data;

assert df.duplicated(subset=['titulo', 'vencimento', 'ask_bid'], keep=False).sum() == 0

In [17]:
# Today's date to register alongside the values of the scraped data;
d0 = dt.date.today()

df['data'] = d0
df.head()

,titulo,taxa,pu,vencimento,ask_bid,data
0,Tesouro Prefixado 2027,"13,18%",766.97,2027-01-01,ask,2024-11-02
1,Tesouro Prefixado 2031,"13,24%",467.73,2031-01-01,ask,2024-11-02
2,Tesouro Prefixado com juros semestrais 2035,"12,97%",878.82,2035-01-01,ask,2024-11-02
3,Tesouro Selic 2027,"SELIC + 0,0547%",15545.75,2027-03-01,ask,2024-11-02
4,Tesouro Selic 2029,"SELIC + 0,1246%",15482.50,2029-03-01,ask,2024-11-02


<h3> Putting On DB Postgres </h3>

<h5> DB Credentials </h5>

In [18]:
from sqlalchemy import text, create_engine

db_txt = 'creds.txt'

with open(db_txt, 'r') as f:
    uri = f.readline()[:-1] # to remove \n


In [19]:
# creating engine with the database url passed
engine = create_engine(uri)

<h5> Create a table to store scraped data </h5>

In [20]:
query = """ CREATE TABLE br_treasury_titles(
                            data date,
                            titulo varchar(64), 
                            taxa varchar(64), 
                            pu numeric(7,2), 
                            vencimento date,
                            ask_bid varchar(16)
                            )"""

cursor = engine.connect()


try:
    cursor.execute(text(query))
    cursor.commit()
    cursor.close()

except:
    print("Table already exists in DB. Drop or switch to another db.")


Table already exists in DB. Drop or switch to another db.


In [21]:
# ADD Constraint for unique title, expire date and day of scraping

with engine.connect() as con:
    
    try:
        con.execute(text("""ALTER TABLE br_treasury_titles ADD CONSTRAINT "uniTiDaVen" UNIQUE (titulo, vencimento, data, ask_bid)"""))
        con.commit()
    except:
        None

In [22]:
# Inserting the Data (load phase); 

with engine.connect() as con:

    try:
        df.to_sql(name = 'br_treasury_titles', 
              con=con,
              schema='public',
              if_exists='append',
              index=False)
        
    except:
        print('there is duplicated data')
        #duplicated = True

<h3> ### </h3>